# Merging two Data-sets
(It took me a long time and a lot of reading about how dask computes the task but i got it working.)
you will need npm, node.js and dask with pandas
https://docs.dask.org/en/latest/install.html

## First we define a localcluster and a client

In [1]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)

## Some info about the client and all of our peons:

In [6]:
client

Client Scheduler: tcp://127.0.0.1:52807 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.51 GB


In [4]:
cluster.workers

{0: <Nanny: tcp://127.0.0.1:52838, threads: 1>,
 1: <Nanny: tcp://127.0.0.1:52835, threads: 1>,
 2: <Nanny: tcp://127.0.0.1:52829, threads: 1>,
 3: <Nanny: tcp://127.0.0.1:52832, threads: 1>}

## Now, here is where the fun begins

For some bizzare reason it took me almost a week to figure out dask.
I don't want to blame the library for it but the fact that dask's errors are vague does not help at all.
90% of the time i was geting a memory error and not much more to work with.

back to the task

## We need to read the tsv files to dataframes very simillar to those used by pandas
Make sure to type in the right name of the file

In [5]:
import dask.dataframe as dask

df1 = dask.read_csv("title.basics.full.tsv",sep='\t',dtype=str)
df1 = df1.set_index('tconst')

df2 = dask.read_csv("title.ratings.full.tsv",sep='\t',dtype=str)
df2 = df2.set_index('tconst')

In [7]:
df1 = client.persist(df1)

## Some lovely information about our datasets:

In [8]:
df1.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tconst,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
df2.head()

,averageRating,numVotes
tconst,,
tt0000001,5.6,1660
tt0000002,6.1,203
tt0000003,6.5,1373
tt0000004,6.2,123
tt0000005,6.2,2161


## This is just one of possible variations on joining data-sets

In [10]:
result = dask.merge(df1, df2, left_on='tconst', right_on='tconst').compute()

## Let's se what we got:

In [11]:
result.head()

,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
tconst,,,,,,,,,,
tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.6,1660
tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.1,203
tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1373
tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",6.2,123
tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2161


look's about right, no column was lost and there is no duplicates to be seen

In [12]:
result.to_csv('all_videos.tsv', sep='\t')

And at long last we finnaly got to the finish line and it took little to no time compared to `merge_tsv_low_memory_use.py`

# Extracting all records of type "movie"

## First let me implement some libs

In [ ]:
import csv
import os

The whole script is pretty simple, I opened a destination file and with that after also opening the 'all_videos.tsv' file i simply went over every single record in it and saved the ones that have a type movie as at least one of them (the script breaks down when we implement a category that has movie in it like "horror movie" it won't treat is as movie and skip it, but i think that based on how dataset is filled it should not happen) 

In [ ]:
with open(os.path.join(os.path.dirname(__file__), 'movies.tsv'), 'w', newline='',encoding='utf8') as result:
    writer = csv.writer(result, delimiter ='\t')
    with open(os.path.join(os.path.dirname(__file__), 'all_videos.tsv'),encoding='utf8') as videos:
        movie_list = videos.readline().split('\t')
        movie_list[len(movie_list)-1] = movie_list[len(movie_list)-1].replace('\n', '')
        writer.writerow(movie_list)
        print(movie_list)
        
        
        for line_videos in videos:
            if line_videos.split('\t')[1] == 'movie':
                movie_list=line_videos.split('\t')
                movie_list[len(movie_list)-1] = movie_list[len(movie_list)-1].replace('\n', '')
                writer.writerow(movie_list)

# Rating all videos by year and by generes 

## We will be using the same set of libs as before so i skipped it this time, if you didn't run it earlier you need to run the first block of the "chapter" above

I need to admit, the code is pretty messy, it's a lot of if's and for's but i found it to be sufficiently fast.
So, i opened both files, the result and the source, splitting them by \t and so on, then i started looking for every line in `all_videos.tsv`
```python
for videos_line in videos:
```
in every line i checked if the genres column is not empty
```python
if videos_line.split('\t')[9]!='' and videos_line.split('\t')[10]!='' :
```
then when that was confirmed i started to look for matching genres in the result file
```python 
for videos_genre in videos_line.split('\t')[8].split(','):
                    found = False
                    
                    for result_element in range(len(result_list)):
```
if the script finds matching genres I need to somehow add the rating to already calculated mean. I did that by multiplying the mean by numbers of elements stored in the last column and then adding new rating to this result and adding one to the total count of accounted records, after that i just reversed the process so i used this formula.
```
total = mean * count
new total = total + new rating
new count = count ++ 
new mean = new total / new count
```
I just squished it into one line so that i can store it in the variable right away because from what i know pyhton is very good at counting but not as good in storing variables every line and that would add a significant amount of processing time in case of very large files.

If scirpt doesnt find a match i just create one real quick and set total count to 1

In [ ]:
with open(os.path.join(os.path.dirname(__file__), 'movie_rating_by_genres.tsv'), 'w', newline='',encoding='utf8') as result:
    writer = csv.writer(result, delimiter ='\t')
    writer.writerow(['genre', 'averageRating', 'totalCount'])

    with open(os.path.join(os.path.dirname(__file__), 'all_videos.tsv'),encoding='utf8') as videos:
        videos.readline()
        result_list = []
        for videos_line in videos:
            if videos_line.split('\t')[9]!='' and videos_line.split('\t')[10]!='' :
                for videos_genre in videos_line.split('\t')[8].split(','):
                    found = False
                    
                    for result_element in range(len(result_list)):
                        
                        if videos_genre == result_list[result_element][0]:
                            result_list[result_element][1]=(float(result_list[result_element][1]) * float(result_list[result_element][2]) + float(videos_line.split('\t')[9]))/(float(result_list[result_element][2])+1.0)
                            result_list[result_element][2] = result_list[result_element][2] + 1
                            found = True
                            
                    if not found:
                        result_list.append([videos_genre, videos_line.split('\t')[9], 1])
                        found = True
                        
        for elem in result_list:
            writer.writerow(elem)

## Both rating scripts work basically the same

The easy part in rating by year is that there will always be only one year per record so i dont need to iterate through it like i did above

In [ ]:
with open(os.path.join(os.path.dirname(__file__), 'movie_rating_by_year.tsv'), 'w', newline='',encoding='utf8') as result:
    writer = csv.writer(result, delimiter ='\t')
    writer.writerow(['year', 'averageRating', 'totalCount'])

    with open(os.path.join(os.path.dirname(__file__), 'all_videos.tsv'),encoding='utf8') as videos:
        videos.readline()
        result_list = []
        for videos_line in videos:
            if videos_line.split('\t')[9]!='' and videos_line.split('\t')[10]!='' :
                found = False
                for result_element in range(len(result_list)):
                    if videos_line.split('\t')[5] == result_list[result_element][0]:
                        result_list[result_element][1]=(float(result_list[result_element][1]) * float(result_list[result_element][2]) + float(videos_line.split('\t')[9]))/(float(result_list[result_element][2])+1.0)
                        result_list[result_element][2] = result_list[result_element][2] + 1
                        found = True
                if not found:
                        result_list.append([videos_line.split('\t')[5], videos_line.split('\t')[9], 1])
                        found = True
        for elem in result_list:
            writer.writerow(elem)

# Plotting the results

## Some good'ol imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

When it comes to ploting, well, bussines as usual, we use matplot lib to save the png of the charts, pandas for reading the dataframes and os for ... well it's just ther for the path...

In [ ]:
a= pd.read_csv(os.path.join(os.path.dirname(__file__), "movie_rating_by_year.tsv"),sep='\t',low_memory=False,memory_map=True)
a.plot.scatter(x='year', y='averageRating')
plt.savefig(os.path.join(os.path.dirname(__file__), 'movie_rating_by_year.png'))
a= pd.read_csv(os.path.join(os.path.dirname(__file__), "movie_rating_by_genres.tsv"),sep='\t',low_memory=False,memory_map=True)
a.plot.scatter(x='genre', y='averageRating', s=20).tick_params(axis='x', rotation=-37,direction= 'in',labelsize=7)

plt.savefig(os.path.join(os.path.dirname(__file__), 'movie_rating_by_genre.png'))